In [1]:
import camelot as cam
import matplotlib.pyplot as plt
import re
import string

# Function to extract table in a document page

In [2]:
def get_table(url, pages, table_areas, flavor = "stream"):
    table = cam.read_pdf(filepath = url, flavor = flavor, pages = pages, table_areas = table_areas)
    return table

In [3]:
path = "/Users/kehindeajayi01/Desktop/My_research/PSU/papers/development-and-exploration-of-refractory-high-entropy-alloysa-review.pdf"


## Applying the function

In [5]:
tab4 = get_table(url = path, pages = "4", table_areas = ['100,630,556,52'])
tab_4 = tab4[0].df

In [129]:
#tab_4.head()

## Selecting rows with appropriate data

In [6]:
"""This function that takes a table and returns phases and alloys"""
def get_phases_alloys(table, column1, column2):
    Phases = table[column1][1:]
    Alloys = table[column2][1:]
    return Phases, Alloys
    
Phases, Alloys = get_phases_alloys(tab_4, column1 = 1, column2 = 0)

#Phases

# Extracting the Phases

In [7]:
def clean_phases(phases):
    new_ph = []
    for ph in phases:
        if ph != "":
            new_ph.append(str(ph))
    return new_ph

# Extracting the Alloys

In [8]:
def clean_alloys(Alloys, Phases):
    new_alloys = []
    clean_ph = clean_phases(Phases)
    for al in Alloys:
        if al == "" or  al.isdigit():
            continue
        else: 
            new_alloys.append(str(al))
    return new_alloys, clean_ph

## Remove reference from the Extracted Alloys

In [9]:

def remove_ref(Alloys, Phases):
    new_alloys, clean_ph = clean_alloys(Alloys, Phases)
    pattern = re.compile('[A-Za-z]*[0-9]*\.[0-9]+$')

    alloys_new = new_alloys.copy()
    index = 0
    while index < len(alloys_new):
        elem = alloys_new[index]
        if pattern.search(elem) is None:
            name = elem.rstrip(string.digits)
            alloys_new[index] = name
        index += 1
    return alloys_new, clean_ph
    

## Tracking phases with length more than 2

In [10]:
def PhasesAbove4(Alloys, Phases):
    phases_res = []
    alloys_res = []
    alloys_rem = []
    phases_rem = []

    alloys_new, new_ph = remove_ref(Alloys, Phases)
    #new_ph = clean_phases(Phases)
    index = 0
    while index < len(new_ph) and index < len(alloys_new):
        phase = new_ph[index]
        alloy = alloys_new[index]
        if len(phase) > 4:
            phases_res.append(phase)
            alloys_res.append(alloy)
        else: 
            phases_rem.append(phase)
            alloys_rem.append(alloy)
        index += 1
    return phases_res, alloys_res, alloys_rem, phases_rem

## Coorecting wrong separators

In [11]:
def CorrectSeparator(Alloys, Phases):
    phases_new, alloys_new, alloys_rem, phases_rem = PhasesAbove4(Alloys, Phases)

    phases_res = []
    index = 0
    while index < len(phases_new):
        phase = phases_new[index].split()
        for i in range(len(phase)):
            if phase[i].isdigit():
                phase[i] = phase[i].replace(phase[i], "+")
        phase = " ".join(phase)
        phases_res.append(phase)
        index += 1
    phases_rem.extend(phases_res)
    alloys_rem.extend(alloys_new)
    return phases_rem, alloys_rem

## Creating dataframe for the Alloys and Phases

In [12]:
import pandas as pd 

def create_dataframe(Alloys, Phases):
    Comb_Phases, Comb_Alloys = CorrectSeparator(Alloys, Phases)
    df = pd.DataFrame(data={"Alloys": Comb_Alloys, "Phases": Comb_Phases})
    return df

df = create_dataframe(Alloys, Phases)
#df = pd.DataFrame(data = {"Alloys": al_rem, "Phases": ph_rem})

# Table 4

In [13]:
df

,Alloys,Phases
0,MoNbTaW,BCC
1,MoNbTaVW,BCC
2,HfNbTaTiZr,BCC
3,MoNbTiVZr,BCC
4,MoNbTiZr,BCC
5,MoNbTiV0.25Zr,BCC
6,MoNbTiV0.5Zr,BCC
7,MoNbTiV0.75Zr,BCC
8,MoNbTiV1.25Zr,BCC
9,MoNbTiV1.5Zr,2BCC


# Table 5

In [13]:
# get table in page 5
tab5 = get_table(url = path, pages = "5", table_areas = ['100,630,556,52'])
tab_5 = tab5[0].df


In [35]:
#tab_5.head()

## Get phases and alloys for table 5

In [14]:
Phases_5, Alloys_5 = get_phases_alloys(tab_5, column1 = 2, column2 = 0)

In [15]:
df2 = create_dataframe(Alloys_5, Phases_5)

In [16]:
df2

,Alloys,Phases
0,CrMoNbTaVW,BCC
1,HfMoNbTaTiVZr,2BCC
2,HfMoTaTiZr,BCC
3,Mo0.1NbTiV0.3Zr,BCC
4,Mo0.3NbTiV0.3Zr,BCC
5,Mo0.3NbTiVZr,BCC
6,Mo0.5NbTiVZr,BCC
7,Mo0.5NbTiV0.3Zr,BCC
8,Mo0.7NbTiV0.3Zr,BCC
9,Mo0.7NbTiVZr,BCC


# Table in Page 6

In [17]:
# get table in page 6
tab6 = get_table(url = path, pages = "6", table_areas = ['100,630,500,200'])
tab_6 = tab6[0].df


In [101]:
#tab_6.head()

In [18]:
Phases_6, Alloys_6 = get_phases_alloys(tab_6, column1 = 2, column2 = 0)

In [19]:
df3 = create_dataframe(Alloys_6, Phases_6)

In [20]:
df3

,Alloys,Phases
0,CoCrMoNbTi0.4,BCC
1,CrMoNbReTaVW,2BCC
2,CrMoNbTaTiVWZr,BCC
3,CrMoNbTaTiVZr,BCC
4,HfNb0.6Ti0.8V0.4Zr1.2,BCC
5,HfMo0.5NbTiV0.5,BCC
6,HfMoTiWZr,BCC
7,MoNbTaTiV,BCC
8,MoNbTaTiVZr,BCC
9,MoNbTaV0.4W1.6,BCC


# Table in Page 14

In [21]:
tab14 = get_table(url = path, pages = "14", table_areas = ['50,425,556,52'])
tab_14 = tab14[0].df

In [128]:
#tab_14

In [22]:
Phases_14, Alloys_14 = get_phases_alloys(tab_14, column1 = 4, column2 = 0)

In [23]:
df4 = create_dataframe(Alloys_14, Phases_14)

In [24]:
df4

,Alloys,Phases
0,AlNbTiV,BCC
1,AlCr0.5NbTiV,BCC
2,AlCrMoTi,BCC
3,AlMoNbTi,BCC
4,AlCrMoNbTi,BCC
5,AlNb1.5Ta0.5Ti1.5Zr0.5,BCC
6,AlNbTa0.5TiZr0.5,B2
7,AlMo0.5NbTa0.5TiZr0.5,B2
8,Al0.3NbTa0.8Ti1.4V0.2Zr1.3,BCC
9,HfMoNbTiZr,BCC


# Table in Page 19

In [14]:
tab19 = get_table(url = path, pages = "19", table_areas = ['50,660,556,50'])
tab_19 = tab19[0].df

In [23]:
#tab_19.head()

In [16]:
Phases_19, Alloys_19 = get_phases_alloys(tab_19, column1 = 2, column2 = 0)

## Case where phases continue in second line

In [17]:
def phase_cleaning(Alloys, Phases):

    index = 1
    phs_2 = Phases.copy()
    while index < len(Alloys) and index < len(phs_2):
        alloy = Alloys[index]
        ph = phs_2[index]
        if alloy == "" and ph != "":
            prev_ph = phs_2[index - 1]
            ph_comb = prev_ph + " " + ph
            phs_2[index - 1] = ph_comb
            phs_2.pop(index)
    
        index += 1
    return phs_2

In [18]:
cleaned_phases = phase_cleaning(Alloys_19, Phases_19)

In [20]:
#cleaned_phases

In [21]:
df5 = create_dataframe(Alloys_19, cleaned_phases)

In [22]:
df5

,Alloys,Phases
0,Al0.5CrMoNbTi,BCC
1,Al0.5CrMoNbTiV,BCC
2,Al0.5CrMoNbV,BCC
3,AlCrMoTiW,BCC
4,Al9.2Cr5.7Hf0.5Mo1.3Nb47.0Ti25.9V9.6W0.8,BCC
5,Al9.2Cr5.7Hf0.5Mo1.3Nb47.0Ti25.9V9.6W0.8,BCC
6,Al9.2Cr5.7Hf0.5Mo1.3Nb47.0Ti25.9V9.6W0.8,BCC
7,AlCrMoTaTi,BCC
8,AlCrMoTaTi,BCC
9,AlCrMoTiW,BCC


In [40]:
tb = cam.read_pdf(path, pages = "20", flavor = "stream")
tb

<TableList n=1>

# Table in Page 20

In [46]:
tab20 = get_table(url = path, pages = "20", table_areas = ['50,660,300,200'])
tab_20 = tab20[0].df

In [28]:
tab20[0]._bbox

(50.0, 50.0, 556.0, 660.0)

In [55]:
nece_rows = tab_20[0][1:4].str.split("\n")
nece_rows

1        [As-cast, BCC, 1100, HfNbTaTiZr105]
2        [HfNbTaTiZr105, As-cast, BCC, 1300]
3    [Annealed, BCC, 1000, Al2Nb3TaTi3Zr106]
Name: 0, dtype: object

In [56]:
Phases = []
Alloys = []

alpha = "ABCDEFGH"
num = "0123456789"
for row in nece_rows:
    for elem in row:
        if str(elem).startswith("BCC"):
            Phases.append(elem)
        elif str(elem).startswith(alpha) and str(elem).endswith(num):
            Alloys.append(elem)
        

In [57]:
Phases, Alloys

(['BCC', 'BCC', 'BCC'], [])